In [1]:
import pandas as pd

file_path='/Users/gimhagyeong/test/B0005_with_lowess_features.csv'
df=pd.read_csv(file_path)

# 기본 피처 (lowess 파생피처 사용 X)

In [4]:
import numpy as np
import plotly.express as px
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler



# 1. 사용할 feature 선택
features = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load'
]
X = df[features]

# 2. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lof = LocalOutlierFactor(
        n_neighbors=20,
        contamination=0.01   # 예상 이상치 비율
    )

y_pred = lof.fit_predict(X_scaled)
    
lof_scores = -lof.negative_outlier_factor_

scores_sorted = np.sort(lof_scores)
    


df['lof_pred'] = y_pred
df['lof_scores'] = lof_scores
df['lof_anomaly']=(df['lof_pred']==1).astype(int)

print("LOF 이상치:", df['lof_anomaly'].sum())

LOF 이상치: 49782
